In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# The federal election 2017
The election for the German Bundestag takes place every four years, usually in September. In 2017, on the 24th September the 19th German Bundestag was to be voted for. 
The government previous to this election was formed by <i>CDU/CSU</i> and <i>SPD</i>. The parties <i>Die Linke</i> and <i> Bündnis 90/Die Grünen</i> (also often referred to as <i>Die Grünen</i>) where part of the parliament as well. 

## The German voting system
The German voting system gives every person, who is allowed to vote, two votes, the first vote and second vote. With the former one votes for a candidate representative for the respective constituency in which one votes. Every candidate that gets the most votes in a constituency becomes a member of the Bundestag. The second vote gives the voter the opportunity to vote explicitly for one party. In a complex calculation the second votes for each party (along with how many direct mandates they won) are used to determine the number of seats a party gets in the Bundestag. Only parties, which get at least five percent of second votes or win a minimum of three direct mandates will be part of the Bundestag. Another peculiarity to notice, the party <i>CDU</i> can only be voted for in federal states other than Bavaria, and only voters in Bavaria can vote for the <i>CSU</i>. Both parties form a parliamentary group in the Bundestag, which is often referred to as <i>Union</i> (<i>CDU/CSU</i> is used as well).

## Election results
In the following we are showing the results of this election based on data from the Bundeswahlleiter [4], the official person/institution in Germany to observe and control the election. Furthermore, we will show some more analysis of the election results, especially with respect to the age structure of both the voters and the candidates voted into the parliament. This analysis will be helpful to form stronger and more justified hypothesis later on. Given that the dataset was collected on twitter and a common hypothesis is, that social media is more commonly used by younger people, the age analysis will give a better context for our findings in comparison to the election results. Additionally, there will be visualizations representing the winning party for the first and second vote in every constituency.
Our analysis will be restricted to the parties that were represented in the 19th Bundestag.

### First and second votes

We start by looking at the number of votes the parties obtained, split into first and second vote. For this purpose we use a grouped bar chart, where for each party the first and second vote were grouped together. This grouping allows us to identify, which party was strong for which vote. And the bar chart itself is well suited to show the differences between the parties proportionally. In the following chart we did not add <i>CDU</i> and <i>CSU</i> together, because people vote for the party and not for the parliamentary group. The color choice is made to take color blindness and similar conditions into account.

In [2]:
data = pd.read_csv('../data/btw17_erg_BL.csv', sep=';')
colors_parties = {''
    'CDU/CSU': '#000000',
    'SPD': '#ec1b23',
    'AFD': '#08c1ff',
    'FDP': '#fff203',
    'DIE LINKE': '#ff08c1', 
    'BÜNDNIS 90/DIE GRÜNEN': '#71be43'
}

In [3]:
relevant_columns_erst = ['CDU_Erststimmen_Endgültig', 'CSU_Erststimmen_Endgültig', 'SPD_Erststimmen_Endgültig', 'AFD_Erststimmen_Endgültig', 'FDP_Erststimmen_Endgültig', 
                         'LINKE_Erststimmen_Endgültig', 'GRUENE_Erststimmen_Endgültig']
relevant_columns_zweit = ['CDU_Zweitstimmen_Endgültig', 'CSU_Zweitstimmen_Endgültig', 'SPD_Zweitstimmen_Endgültig', 'AFD_Zweitstimmen_Endgültig', 'FDP_Zweitstimmen_Endgültig',
                          'LINKE_Zweitstimmen_Endgültig', 'GRUENE_Zweitstimmen_Endgültig']
partein_order = ['CDU', 'CSU', 'SPD',  'AfD', 'FDP', 'DIE LINKE', 'GRÜNE', ]

erststimmen = []
for col in relevant_columns_erst:
    erststimmen.append(int(data[data['BundeslandID']==99][col]))

zweitstimmen = []
for col in relevant_columns_zweit:
    zweitstimmen.append(int(data[data['BundeslandID']==99][col]))

In [8]:
fig = go.Figure(data=[
    go.Bar(name='first vote', x=partein_order, y=erststimmen, marker_color='orange'),
    go.Bar(name='second vote', x=partein_order, y=zweitstimmen, marker_color='grey') #direct labeling to confussing because of the big numbers
])

fig.update_layout(plot_bgcolor="white", 
                  title={
                    'text': "Number of first and second votes per party", 
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                  legend={
                    'yanchor': "top",
                    'y': 0.99,
                    'xanchor': "left",
                    'x': 0.92}
                  )
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#efefef', zeroline=True, title='Amount of votes')
fig.update_xaxes(title='Parties')
fig.show()

The <i>CDU</i> (along with the <i>CSU</i>) won this election. The second most votes went to the SPD. The grouping of first and second votes per party shows, that the <i>SPD</i>, <i>CDU</i> and <i>CSU</i> got more first votes than second votes. For the other parties it is the opposite. Especially for the <i>FDP</i> the gap between the number of first and second votes obtained is quite big with almost 1.8 million more second votes.

### Seat distribution in the Bundestag

Based on the votes obtained the number of seats a party gets is calculated. The following visualization shows the number of seats obtained by each party, here <i>CSU</i> and <i>CDU</i> are grouped together, since they form a parliamentary group. A usual way of representing the number of seats in a (German) parliament is a (half) pie chart. Even though this type of a chart is usually not a good way of representing data, since it can be hard to compare the sizes of the segments and thus the identification of smaller differences between categories is more challenging, it can be well justified in this context. The aim of looking at the number of seats a parliamentary group holds is directly connected to the question which groups form a coalition or can get majorities. A pie chart with direct labeling of the number of seats allows the viewer to easier identify which coalitions are possible compared to other types of charts and is thus best suited for the data at hand.
Our color choice is based on the official colours of the parties, since this will be most intuitive to the majority of possible viewers of the plots. However, this may not be the optimal solution in terms for people with color deficiencies, but we checked (using https://www.color-blindness.com/coblis-color-blindness-simulator/) how this would effect different conditions and found, that for most color deficiencies the choosen colors work well. Thus we decided to use the official color of each party.  

In [9]:
sitzverteilung = pd.DataFrame({'Partei': ['CDU/CSU', 'SPD', 'AfD', 'FDP', 'DIE LINKE',  'BÜNDNIS 90/DIE GRÜNEN'], 
                               'Sitze': [246, 153, 94, 80, 69, 67]})

In [10]:
fig = go.Figure(data=[go.Pie(labels=sitzverteilung['Partei'], values=sitzverteilung['Sitze'], textinfo='value')])

fig.update_traces(marker=dict(colors=list(colors_parties.values()), line=dict(color='#ffffff', width=2))) 
fig.update_layout(plot_bgcolor="white", 
                  title={
                    'text': "Number of seats in the 19th Bundestag", 
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}
                  )
fig.add_annotation(x=1, y=1,
                    text="Number of seats needed<br> for a simple majority: 355",
                    showarrow=False,
                  )

fig.show()

Possible coalitions (given the political circumstance that no party wants to work together with the <i>AfD</i>) that can be seen in this are for example <i>SPD</i> and <i> CDU/CSU</i> <i> CDU/CSU, Grüne</i> and <i>Linke</i> or <i> CDU/CSU, Grüne</i> and <i>FDP</i>. Note, that this pie chart is based on the party a member of parliament got voted in for, it does not account for some politicians who later on in the legislative period left a parliamentary group and became individiual members of parliament.

### Election results broken down by age of the voter

After every election an often asked question is about which age groups preferred which party. We look at it in this context chapter, because our dataset was recorded on twitter, which is not used by every age group to the same extend. <br>
Again we will use a grouped bar chart since it allows us to compare different age groups along with the voting behavior of each of these groups.

Note beforehand: The unequal grouping of the age groups in terms of size is due to the data processing of the Bundeswahlleiter. Additionally, there is no other data openly available for that purpose from trustworthy sources. Thus, in the following charts one has to be more careful with interpretations.
<br>

In [11]:
age_voters = pd.read_csv('../data/btw17_erg_nachAlter_erstUzweit_Bund.csv', sep=';')
#age_voters

In [12]:
partei = ['CDU', 'CSU', 'SPD', 'AfD', 'FDP', 'DIE LINKE', 'GRÜNE']
partei_norm = [p + '_norm' for p in partei]

age_voters_norm = [[age_voters[p]/age_voters['Summe']] for p in partei]

#normalization of the values (based on how many people in that group voted) to get rid of the number inbalances between the age groups
age_voters['CDU_norm'] = age_voters['CDU']/age_voters['Summe']
age_voters['CSU_norm'] = age_voters['CSU']/age_voters['Summe']
age_voters['SPD_norm'] = age_voters['SPD']/age_voters['Summe']
age_voters['AfD_norm'] = age_voters['AfD']/age_voters['Summe']
age_voters['FDP_norm'] = age_voters['FDP']/age_voters['Summe']
age_voters['DIE LINKE_norm'] = age_voters['DIE LINKE']/age_voters['Summe']
age_voters['GRÜNE_norm'] = age_voters['GRÜNE']/age_voters['Summe']

age_voters_l = [[age_voters.iloc[i][p] for p in partei] for i in range(1, 7)]

#age_voters_norm_l = [[age_voters.iloc[i][p]/age_voters['Summe'][i] for p in partei] for i in range(1, 7)] #look whether i corrected it the right way
age_voters_norm_l = [[age_voters.iloc[i][p] for p in partei_norm] for i in range(1, 7)]

age_voters['Geburtsjahresgruppe'] = age_voters['Geburtsjahresgruppe'].replace(['1947 und früher'],'1947 and before')

In [16]:
age_generations = ['1993-1999', '1983-1992', '1973-1982', '1958-1972', '1948-1957', '1947 und before']
fig = go.Figure(data=[
    go.Bar(name='CDU', x=age_generations, y=age_voters['CDU_norm'][1:7], marker_color=colors_parties['CDU/CSU']),
    go.Bar(name='CSU', x=age_generations, y=age_voters['CSU_norm'][1:7], marker_color='darkgray'),
    go.Bar(name='SPD', x=age_generations, y=age_voters['SPD_norm'][1:7], marker_color=colors_parties['SPD']),
    go.Bar(name='AfD', x=age_generations, y=age_voters['AfD_norm'][1:7], marker_color=colors_parties['AFD']),
    go.Bar(name='FDP', x=age_generations, y=age_voters['FDP_norm'][1:7], marker_color=colors_parties['FDP']),
    go.Bar(name='DIE LINKE', x=age_generations, y=age_voters['DIE LINKE_norm'][1:7], marker_color=colors_parties['DIE LINKE']),
    go.Bar(name='GRÜNE', x=age_generations, y=age_voters['GRÜNE_norm'][1:7], marker_color=colors_parties['BÜNDNIS 90/DIE GRÜNEN']),

])

fig.update_layout(plot_bgcolor="white",
                  title={
                    'text': 'First vote distribution per age group', 
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                  legend={
                      'title': 'Party'
                  }
                  )
    
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#efefef', zeroline=True, title_text='Percentage')
fig.update_xaxes(title_text='Age group')

fig.show()

The visualization above shows multiple things about the first votes. Firstly, the <i>CDU</i> got the most votes in each of the age groups with the <i>SPD</i> being the second most voted for party. Whereas the <i>SPD</i> scored around 25% of votes in each of the formed age groups, the <i>CDU</i> got more votes with the increasing ages of the voters. Secondly, it can be observed, that <i>Die Grünen</i> have more voters in younger age groups than older age groups. The same holds true for <i> Die Linke</i>, but they have more voters in older age groups than <i> Die Grünen </i> have. The votes obtained by the <i>FDP</i> get slightly less for older age groups. Compared to that the <i>AfD</i> has the most voters in middle-aged age groups. Interestingly, when ignoring <i>SPD</i> and <i> CDU </i>, <i>CSU</i> the order of the parties based on the amount of votes is reversed between the youngest and oldest age group. 

In [18]:
age_generations = ['1993-1999', '1983-1992', '1973-1982', '1958-1972', '1948-1957', '1947 and before']
fig = go.Figure(data=[
    go.Bar(name='CDU', x=age_generations, y=age_voters['CDU_norm'][8:14], marker_color=colors_parties['CDU/CSU']),
    go.Bar(name='CSU', x=age_generations, y=age_voters['CSU_norm'][8:14], marker_color='darkgray'),
    go.Bar(name='SPD', x=age_generations, y=age_voters['SPD_norm'][8:14], marker_color=colors_parties['SPD']),
    go.Bar(name='AfD', x=age_generations, y=age_voters['AfD_norm'][8:14], marker_color=colors_parties['AFD']),
    go.Bar(name='FDP', x=age_generations, y=age_voters['FDP_norm'][8:14], marker_color=colors_parties['FDP']),
    go.Bar(name='DIE LINKE', x=age_generations, y=age_voters['DIE LINKE_norm'][8:14], marker_color=colors_parties['DIE LINKE']),
    go.Bar(name='GRÜNE', x=age_generations, y=age_voters['GRÜNE_norm'][8:14], marker_color=colors_parties['BÜNDNIS 90/DIE GRÜNEN']),

])

fig.update_layout(plot_bgcolor="white",
                  title={
                    'text': 'Second vote distribution per age group', 
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                  legend={
                      'title': 'Party'
                  }
                  )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#efefef', zeroline=True,
                 title_text='Percentage')
fig.update_xaxes(title_text='Age group')

fig.show()

This grouped bar chart shows the behavior of second votes, which can be compared to the chart above to see some similarities between the voting behavior for first and second votes. For the second vote the general observations from above still hold true, but neither <i>SPD</i> nor <i>CDU</i> have as big of a lead as with the first votes, but they are still the most voted for parties. The biggest knowledge gain from comparing the charts for first and second votes is, that more voters supported smaller parties with their second vote compared to their first vote. To put the results of this into perspective it is important to be aware of the different amounts of people per age group. The bar chart below shows this visually. Unsurprisingly the age group 1958-1972 had the most number of active voters for the 2017 election. Nonetheless a clear trend, that more people are in older age groups is visible. The in comparison low number of voters of the youngest age group can partly be explained by the range of birth years being the samllest for the age group 1993-1996 as well as by the baby boomer generation, that is people being born between mid 1950s to end of 1960s. In context of the charts shown above, this means that the depicted voting behavior of older age groups has a bigger impact on the overall result than that of younger age groups.

In [20]:
fig = px.bar(x=age_voters[1:7]["Geburtsjahresgruppe"], y=age_voters[1:7]["Summe"], 
              labels={'x': 'Age group',
                      'y': 'Number of active voters'}
              )

fig.update_layout(plot_bgcolor="white",
                  title={
                    'text': 'Active voters per age group', 
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}
                  )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#efefef')

fig.show()

### Voter turnout per age group

Not only the number of eligible or active voters per age group is important for the weighting of the age group specific voting behavior, but also the voter turnout. In total 76.2 percent of people, who were allowed to vote, voted in the 2017 election. However, this number differed between age groups. We choose to depict this as a line chart. Even though we have categorical data, we are specifically aiming at seeing a trend between the age groups, and thus decided to use a line chart with marked data points. Due to the date being non-continuous, the lines have the purpose of showing the pattern more clearly. To indicate the discrete data points, we added circle-markers at the respective locations. Additionally we depict the overall voter turnout as another line, to simplify the comparison to it.

Note, here again we have not equally spaced age groups due to the data source with no other suitable and trustworthy data being available, but the age groups are not all the same as above (even though the data comes from the same source).

In [21]:
voter_turnout = pd.read_csv('../data/btw17_wahlbet_nachAlter_Bund.csv', sep=';')
voter_turnout['Geburtsjahresgruppe'] = voter_turnout['Geburtsjahresgruppe'].replace(['1947 und früher'],'1947 and before')

In [24]:
fig = make_subplots()

fig.add_trace(
    go.Scatter(x=voter_turnout[1:]["Geburtsjahresgruppe"], y=[voter_turnout.iloc[0]["Wahlbeteiligung"]] *len(voter_turnout[1:]["Geburtsjahresgruppe"]),
               name='overall voter turnout', mode='lines', line_color='grey')
)

fig.add_trace(
    go.Scatter(x=voter_turnout[1:]["Geburtsjahresgruppe"], y=voter_turnout[1:]["Wahlbeteiligung"], 
               name='voter turnout per age group', line_color='orange'), 
)

fig.update_xaxes(
        title_text = "Age group")

fig.update_yaxes(
        title_text = "Voter turnout in percent",
        showgrid=True, gridwidth=1, gridcolor='#efefef')

# Add figure title
fig.update_layout(plot_bgcolor="white",
                  title={
                    'text': 'Voter turnout per age group', 
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}
                  )

fig.show() 

This chart shows the motivation of first-time voters to vote, at least in comparison to people roughly 10 years older than them. Other than that older people have a higher voter turnout. For people roughly 40 years old at the time of the election the overall voter turnout and the age specific voter turnout are almost the same. As we saw above the age group, that was born in 1947 or before that is the second largest one, but their voter turnout was smaller than for the previous age groups. If we add this to our gained knowledge about the voting behavior of different age groups and the number of people per age group, we can now say, that older age groups impact the overall election result more.

### Age distribution of the members of the Bundestag

Not only the age of the voters can be interesting for formulating more justified hypothesis later on, but the age distribution of the members of parliament as well. Thus, in the following we will have a short look at this. Given that it is an already much discussed topic we are not going into detail and only present this here to give a more comprehensive context for our analysis and visualizations.
First we are visualizing the overall age distribution of the parliament without parliamentary group specific information. For the following visualizations we used histograms, because they are well suited for comparing age ranges. We choose a bin width of five years in order to have a useful representation. The ages where calculated for the year of the election.

In [25]:
mdb = pd.read_csv('../data/btw17_gewaehlteMDB.csv', sep=";")
#mdb

In [26]:
# add additional columns/features
#fraktion since CDU/CSU are seperate parties but one parliamentary party
fraktion = ['CDU/CSU' if fraktion=='CDU' or fraktion=='CSU' else fraktion for fraktion in mdb['Partei_KurzBez']]
mdb['Fraktion'] = fraktion

#agew when member of parliament where elected
age = [2017 - bday for bday in mdb['Geburtsjahr']]
mdb['Alter'] = age

#binning the age
#min age: 25, max age: 77
cut_labels = ['<30', '30-39', '40-49', '50-59', '60-69', '70-79']
cut_bins = [i for i in range(20, 81, 10)]
mdb['Alter_Bin'] = pd.cut(mdb['Alter'], bins=cut_bins, labels=cut_labels)

In [27]:
#Alter nach Fraktion, Anzahl
partei_kurz = list(set(mdb['Fraktion']))

alter_l = []
alter_count_l = []
partei_l = []
for partei in partei_kurz:
    alter_temp = list(mdb[mdb['Fraktion'] == partei]['Alter_Bin'])#['Alter'])
    alter_count = [alter_temp.count(alter) for alter in list(set(alter_temp))]
    alter = list(set(alter_temp))
    partei_l.append([partei for elem in alter_count])
    alter_l.append(alter)
    alter_count_l.append(alter_count)

alter_flat = [alter for partei in alter_l for alter in partei]
alter_count_flat = [count for partei in alter_count_l for count in partei]
partei_flat = [p for partei in partei_l for p in partei]

alter_partei = pd.DataFrame({
    'Partei': partei_flat,
    'Alter': alter_flat,
    'Anzahl': alter_count_flat
})

In [29]:
fig = px.histogram(age, nbins=11,
                   labels={'value':'Age'}
                   )

fig.update_layout(plot_bgcolor="white",
                  title={
                    'text': 'Age distribution in the Bundestag', 
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                  showlegend=False
                  )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#efefef', zeroline=True, title='Count')

fig.show()

This histogram, used to depict the distribution of age among the members of parliament, shows that roughly half of the members of parliament are 50 years or older. We can confirm this by calculating the average age, which was 49.6 years at the time of election. <br>
A look at the parliamentary group specific histograms shows some differences in the age composition. Most notably, the parliamentary groups of <i>FDP</i> and <i>Die Grünen</i> having more younger members while <i>CDU/CSU </i> and <i> AfD </i> have an older age structure. For better comparison between the histograms we used percentages and not total values.

In [33]:
#make subplots
fig = make_subplots(
    rows=2, cols=3,
    shared_yaxes=True,
    shared_xaxes=True,
    #vertical_spacing=0.03,
)

fig.add_trace(
    go.Histogram(
        x=mdb[mdb['Partei_KurzBez']=='GRÜNE']['Alter'],
        histnorm ='percent',
        xbins=dict( # bins used for histogram
            start=20,
            end=80,
            size=5
        ),
        autobinx = False,
        name="Grüne", 
        marker_color=colors_parties['BÜNDNIS 90/DIE GRÜNEN'],
    ),
    row=2, col=3
)

fig.add_trace(
    go.Histogram(
        x=mdb[mdb['Partei_KurzBez']=='DIE LINKE']['Alter'],
        histnorm ='percent',
        xbins=dict( # bins used for histogram
            start=20,
            end=80,
            size=5
        ),
        autobinx = False,
        name="Die LINKE",
        marker_color=colors_parties['DIE LINKE'],
    ),
    row=2, col=2
)

fig.add_trace(
    go.Histogram(
        x=mdb[mdb['Partei_KurzBez']=='FDP']['Alter'],
        histnorm ='percent',
        xbins=dict( # bins used for histogram
            start=20,
            end=80,
            size=5
        ),
        autobinx = False,
        name="FDP",
        marker_color=colors_parties['FDP'],
    ),
    row=2, col=1
)

fig.add_trace(
    go.Histogram(
        x=mdb[mdb['Partei_KurzBez']=='AfD']['Alter'],
        histnorm ='percent',
        xbins=dict( # bins used for histogram
            start=20,
            end=80,
            size=5
        ),
        autobinx = False,
        name="AfD",
        marker_color=colors_parties['AFD'],
    ),
    row=1, col=3
)

fig.add_trace(
    go.Histogram(
        x=mdb[mdb['Partei_KurzBez']=='SPD']['Alter'],
        histnorm ='percent',
        xbins=dict( # bins used for histogram
            start=20,
            end=80,
            size=5
        ),
        autobinx = False,
        name="SPD",
        marker_color=colors_parties['SPD'],
    ),
    row=1, col=2
)

fig.add_trace(
    go.Histogram(
        x=list(mdb[mdb['Partei_KurzBez']=='CDU']['Alter'])+list(mdb[mdb['Partei_KurzBez']=='CSU']['Alter']),
        histnorm ='percent',
        xbins=dict( # bins used for histogram
            start=20,
            end=80,
            size=5
        ),
        autobinx = False,
        name="CDU/CSU",
        marker_color=colors_parties['CDU/CSU'],
    ),
    row=1, col=1
)

fig.update_layout(plot_bgcolor="white",
                  title={
                    'text': 'Age distribution per party', 
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                    height=800,
                    showlegend=True,
                    yaxis_title_text='Percentage',
                    legend={
                        'title': 'Party'
                    }
                    )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#efefef')#, title_text='percentage')
fig.update_xaxes(title_text="Age", row=2, col=2)

fig.show()

This chapter presented a recap of the election results. Besides the winners, it is important to keep in mind, that the voting behaviour between the age groups differs as well as the age groups having different impacts on the end results.

Overall when looking at the topics and networks visible in the #btw17-dataset, it will be interesting to compare this to the actual results. Thus, this chapters main aim was to connect the analysis of twitter as a network for political discussion and communication with the election results to better understand how these two could be connected and how representative of one another they are.